In [14]:
import pandas as pd
import numpy as np
import os
import tifffile as tiff
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, Add, Flatten, MaxPooling2D, Input, Concatenate, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Nadam, Adam
from tensorflow.keras.regularizers import l2, l1

# Load the dataset
df = pd.read_csv("datasets/train.csv")

print(f'Number of features: {df.shape[1]}')
print(f'Number of instances: {df.shape[0]}')
df.head()

# Preprocess numerical data
numerical_features = df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]

# Numerical data scaling
scaler = StandardScaler()
numerical_features = scaler.fit_transform(numerical_features)

# Function to load and preprocess image data
def load_and_preprocess_image(filepath):
    img = tiff.imread(filepath)
    img_array = np.array(img)
    img_array = img_array / 65535.0   # Normalize pixel values
    return img_array

# Load image data
image_data = np.array([load_and_preprocess_image(os.path.join('./train/', filename)) for filename in df['file_name_l1']])

# Target variable
target = df['value_550'].values

# Split data into training, validation, and testing sets
X_train_num, X_temp_num, X_train_img, X_temp_img, y_train, y_temp = train_test_split(numerical_features, image_data, target, test_size=0.3, random_state=42)
X_val_num, X_test_num, X_val_img, X_test_img, y_val, y_test = train_test_split(X_temp_num, X_temp_img, y_temp, test_size=0.5, random_state=42)

# Define the CNN and dense model
class AOTModel:
    def __init__(self, image_shape=(19, 19, 13), num_numerical_features=7):
        # Image processing Neural Network
        self.image_input = Input(shape=image_shape)
        # image_processing_network = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(self.image_input)
        # image_processing_network = BatchNormalization()(image_processing_network)
        # image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        # image_processing_network = Dropout(0.25)(image_processing_network)

        # # image_processing_network = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01))(self.image_input)
        # # image_processing_network = BatchNormalization()(image_processing_network)
        # # image_processing_network = GlobalAveragePooling2D()(image_processing_network)
        # # image_processing_network = Dropout(0.25)(image_processing_network)

        # image_processing_network = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(image_processing_network)
        # image_processing_network = BatchNormalization()(image_processing_network)
        # image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        # image_processing_network = Dropout(0.25)(image_processing_network)

        # # image_processing_network = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01))(image_processing_network)
        # # image_processing_network = BatchNormalization()(image_processing_network)
        # # image_processing_network = GlobalAveragePooling2D()(image_processing_network)
        # # image_processing_network = Dropout(0.25)(image_processing_network)

        # image_processing_network = Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(image_processing_network)
        # image_processing_network = BatchNormalization()(image_processing_network)
        # image_processing_network = GlobalAveragePooling2D()(image_processing_network)
        # image_processing_network = Dropout(0.5)(image_processing_network)

        x = Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(self.image_input)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l1(0.01))(x)
        x = BatchNormalization()(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
    
        residual = Conv2D(64, (1, 1), activation='relu')(self.image_input)
        residual = BatchNormalization()(residual)
        residual = GlobalAveragePooling2D()(residual)
        x = Add()([x, residual])

        # # Numerical processing Neural Network
        self.numerical_input = Input(shape=(num_numerical_features,))
        # numerical_processing_network = Dense(128, activation='relu')(self.numerical_input)
        # numerical_processing_network = BatchNormalization()(numerical_processing_network)
        # numerical_processing_network = Dropout(0.5)(numerical_processing_network)

        # numerical_processing_network = Dense(128, activation='relu')(self.numerical_input)
        # numerical_processing_network = BatchNormalization()(numerical_processing_network)
        # numerical_processing_network = Dropout(0.5)(numerical_processing_network)
        
        # numerical_processing_network = Dense(32, activation='relu')(numerical_processing_network)
        # numerical_processing_network = BatchNormalization()(numerical_processing_network)
        # numerical_processing_network = Dropout(0.5)(numerical_processing_network)

        y = Dense(64, activation='relu')(self.numerical_input)
        y = BatchNormalization()(y)
        y = Dropout(0.5)(y)

        y = Dense(16, activation='relu')(y)
        y = BatchNormalization()(y)
        y = Dropout(0.5)(y)
        
        # Concatenation of both networks
        # aot_network = Concatenate()([image_processing_network, numerical_processing_network])
        # aot_network = Dense(64, activation='relu')(aot_network)
        # aot_network = Dropout(0.5)(aot_network)
        # aot_network = Dense(1)(aot_network)

        concatenated = Concatenate()([x, y])
        z = Dense(32, activation='relu')(concatenated)
        z = Dropout(0.5)(z)
        output = Dense(1)(z)

        # self.aot_network_arquitecture = aot_network
        self.aot_network_arquitecture = output
        # del image_processing_network, numerical_processing_network, aot_network
        del x, y, output

    def model(self):
        model = Model(inputs= [self.image_input, self.numerical_input], outputs=self.aot_network_arquitecture)
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        # optimizer = Nadam(learning_rate=0.001)
        # model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
        return model

# Instantiate the model
model = AOTModel()
model = model.model()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
history = model.fit(
    [X_train_img, X_train_num], y_train,
    validation_data=([X_val_img, X_val_num], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

model.save('aot_model.keras')

# Evaluate the model
val_loss, val_mae = model.evaluate([X_val_img, X_val_num], y_val)
print(f'Validation MAE: {val_mae}')

# Evaluate on test set
test_loss, test_mae = model.evaluate([X_test_img, X_test_num], y_test)
print(f'Test MAE: {test_mae}')

Number of features: 10
Number of instances: 10438
Epoch 1/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 40s 85ms/step - loss: 13.8304 - mae: 1.2111 - val_loss: 8.5434 - val_mae: 0.1000 - learning_rate: 0.0010
Epoch 2/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 8.1265 - mae: 0.4801 - val_loss: 5.3133 - val_mae: 0.0962 - learning_rate: 0.0010
Epoch 3/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 4.7813 - mae: 0.2848 - val_loss: 2.9318 - val_mae: 0.0916 - learning_rate: 0.0010
Epoch 4/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.6350 - mae: 0.1982 - val_loss: 1.7012 - val_mae: 0.0899 - learning_rate: 0.0010
Epoch 5/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.5857 - mae: 0.1467 - val_loss: 1.0769 - val_mae: 0.0949 - learning_rate: 0.0010
Epoch 6/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.9435 - mae: 0.1115 - val_loss: 0.6504 - val_mae: 0.0957 - learning_rate: 0.0010
Epoch 7/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.6113 - mae: 0.1016 - val_l

In [4]:
# Load the saved model
model = load_model('aot_model.keras')

# Load the new dataset
new_df = pd.read_csv("datasets/test.csv")

# Preprocess numerical data
new_numerical_features = new_df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]
new_numerical_features = scaler.transform(new_numerical_features)

# Load and preprocess new image data
new_image_data = np.array([load_and_preprocess_image(os.path.join('./test/', filename)) for filename in new_df['file_name_l1']])

# Predict values for the new data
predictions = model.predict([new_image_data, new_numerical_features])

# Save the predictions to a CSV file
results = pd.DataFrame({
    'id': new_df['id'],
    'value_550': predictions.flatten()
})
results.to_csv('predictions.csv', index=False)

print(results.head())   

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
   id  value_550
0   3   0.136557
1  25   0.192995
2  26   0.083611
3  27   0.096260
4  29   0.104822
